[View in Colaboratory](https://colab.research.google.com/github/J-Fo-S/models/blob/master/pix2pix.ipynb)

# pix2pix

The original pix2pix technique was described in [Image-to-Image Translation with Conditional Adversarial Networks](https://arxiv.org/pdf/1611.07004v1.pdf) and after the initial paper was published there was an outpouring of implementations in a variety of languages and frameworks developed by the community.

We will be using Christopher Hesse's [pix2pix-tensorflow](https://github.com/affinelayer/pix2pix-tensorflow) implementation.

## Setup

In [3]:
# download repository to this machine
!git clone https://github.com/affinelayer/pix2pix-tensorflow.git

Cloning into 'pix2pix-tensorflow'...
remote: Counting objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 13.33 MiB | 54.58 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [4]:
# chaeck that it indeed has been downloaded
!ls

datalab  pix2pix-tensorflow


In [0]:
# the Linux cd command unfortunately does not persist throughout the notebook
!cd pix2pix-tensorflow

In [6]:
!ls

datalab  pix2pix-tensorflow


In [0]:
import os

# you need to change directory in Python for the effect to persist in the kernel
os.chdir('pix2pix-tensorflow')

In [8]:
# it worked!
!ls

docker	docs  LICENSE.txt  pix2pix.py  README.md  server  tools


In [9]:
# use Hesse's utility to download a dataset of facades
!python tools/download-dataset.py facades

downloading https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz
extracting
done


## Train and Test the Model

In [10]:
%%bash

# train the model on the facades dataset
python pix2pix.py \
  --mode train \
  --output_dir facades_train \
  --max_epochs 1 \
  --input_dir facades/train \
  --which_direction BtoA

Process is terminated.


In [0]:
%%bash

# evaluate/test the trained model
python pix2pix.py \
  --mode test \
  --output_dir facades_test \
  --input_dir facades/val \
  --checkpoint facades_train

In [0]:
# output is an HTML file listing the images
!cat facades_test/index.html

In [0]:
from IPython.display import HTML, Image

# we can display HTML inline in the notebook
# but the actual images don't display >.<
HTML(filename='facades_test/index.html')

## Download Results

We can download out results and open them locally

In [0]:
# zip up result directory to download

!zip -r squash.zip facades_test # zip output to download

In [0]:
from google.colab import files

# use colab library to download zip locally
files.download('squash.zip')

# Train on New Images

The format that the library expects is the same as in the original paper. Each example is a single image that is composed of two composite images, each the same size (width and height), sandwiched next to each other with image `A` on the left and image `B` on the right.

![](https://github.com/affinelayer/pix2pix-tensorflow/raw/master/docs/ab.png)

In [0]:
Image('facades/train/213.jpg')

## Upload Files

To upload files in the notebook, we can use the Colab library. The full documentation on working with files in Colaboratory can be found in the [External Data notebook](https://colab.research.google.com/notebooks/io.ipynb).

In [0]:
# an upload file dialog should open
uploaded = files.upload()

In [0]:
# the files are uploaded as binary data in a dictionary
uploaded

In [0]:
# since we are using pix2pix as a commandline utility
# we need to write this data to the file system
open('comb.zip', 'wb').write(uploaded['comb.zip'])

In [0]:
# check that the file was correctly written
!ls

In [0]:
# unzip the archive of uploaded images
!unzip comb.zip

In [0]:
%%bash

# retrain pix2pix on our new images
python pix2pix.py \
  --mode train \
  --output_dir facades_train \
  --max_epochs 200 \
  --input_dir comb \
  --which_direction AtoB

In [0]:
%%bash

# test the new model
python pix2pix.py \
  --mode test \
  --output_dir mario_gen \
  --input_dir comb \
  --checkpoint facades_train

In [0]:
!zip -r squash.zip mario_gen # zip output to download

In [0]:
files.download('squash.zip')